# Initialize and Authenticate

In [ ]:
# AUTHENTICATE TO EARTH ENGINE using your individual Google Earth account credentials
!earthengine authenticate 

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=2WdPUaw6oU42nwlO6XgTwlWCYM61M0nIUchi_aF5VMU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g66nUk4oxQDBGoXXTOxPYMqvG_qXd_sP1wg6P-59vm8jTm-IVe0lQg

Successfully saved authorization token

In [ ]:
# IMPORT & INITIALIZE ee library
import time
import ee
ee.Initialize()

In [ ]:
# IMPORT & MOUNT midscwa gDrive so we can access the .csv with CWA determinations
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Sentinel 2 Data

## Cloud Masking with s2Cloudless
https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

#### Step 1 : Define collection & cloud mask parameters
CLOUD MASK PARAMETERS \
Parameter | Type	| Description \
AOI	ee.Geometry	Area of interest \
START_DATE	string	Image collection start date (inclusive) \
END_DATE	string	Image collection end date (exclusive) \
CLOUD_FILTER	integer	Maximum image cloud cover percent allowed in image collection \
CLD_PRB_THRESH	integer	Cloud probability (%); values greater than are considered cloud \
NIR_DRK_THRESH	float	Near-infrared reflectance; values less than are considered potential cloud shadow \
CLD_PRJ_DIST	float	Maximum distance (km) to search for cloud shadows from cloud edges \
BUFFER	integer	Distance (m) to dilate the edge of cloud-identified objects \

In [ ]:
# Define area of interest (AOI)
center_lat = 40.89038
center_lon = -81.47982 
center = [center_lat, center_lon]
center_reverse = [center_lon, center_lat]
AOI = ee.Geometry.Point(center_reverse)

# Define date range
START_DATE = '2018-08-01' # Selecting data corresponding to wet/winter months
END_DATE = '2019-04-01'

# Define remaining cloud mask parameters
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

#### Step 2: Build Image Collection
Sentinel-2 surface reflectance and Sentinel-2 cloud probability are two 
different image collections. Each collection must be filtered similarly 
(e.g., by date and bounds) and then the two filtered collections must 
be joined on the system:index property.
The result is a copy of the SR collection where each image has a 
new 's2cloudless' property whose value is the corresponding s2cloudless image.

In [ ]:
# Define functions necessary to filter the SR and s2cloudless collections 
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by 
    # the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

# Build an image collection according to the parameters defined above.
s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

#### Step 3: Define Cloud Mask Component Functions

In [ ]:
# Define a function to add the s2cloudless probability layer and derived cloud mask as bands to an S2 SR image input.
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))


# Define a function to add dark pixels, cloud projection, and identified shadows as bands to an S2 SR image input.
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))


# Define a function to assemble all of the cloud and cloud shadow components and produce the final mask.
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)


#### Step 4: Define Cloud Mask Application Functions

In [ ]:
# Define a function to apply the cloud mask to each image in the collection.
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)


# Define function to process the collection
# Add cloud and cloud shadow component bands to each image and 
# then apply the mask to each image. 
# Reduce the collection by median 

def s2_sr_median_func(lat, lon, buffer_m):
  AOI = ee.Geometry.Point([lon, lat])#.buffer(res).bounds()
  s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)
  s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())
  return s2_sr_median

# SRTM Data

In [ ]:
# Extract NASA SRTM v3 Digital Elevation 30m. This dataset has been corrected for voids using open-source data
srtm = ee.Image('USGS/SRTMGL1_003')
srtm.bandNames().getInfo()

['elevation']

In [ ]:
# Generate SRTM layer for slope
slope = ee.Terrain.slope(srtm)
slope.bandNames().getInfo()

['slope']

# JRC Data

In [ ]:
# Extract JRC Global surface Waters Mapping Layers bands of interest
jrc = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
jrc = jrc.select("seasonality", "transition", "max_extent")
jrc.bandNames().getInfo()

['seasonality', 'transition', 'max_extent']

# Functions to Generate Spectral Indices

Band 1 : Coastal Aerosol  
Band 2 : Blue  
Band 3 : Green  
Band 4 : Red  
Band 8 :  NIR  
Band 11 : SWIR  
 

In [ ]:
# NDWI - Normalized Difference Water Index = (Green-NIR)/(Green+NIR)
def make_ndwi(image, green='B3', nir='B8'):
  return image.normalizedDifference([green, nir])  


# NDVI - Normalized Difference Vegetation Index = (NIR-Red)/(NIR+Red)
def make_ndvi(image, red='B4', nir='B8'):
  return image.normalizedDifference([nir, red])  


# MNDVI - Modified Normalized Vegetation Index = (NIR-Red)/(NIR+Red-2*Aerosol)
def make_mndvi(image, red='B4', nir='B8'):
  nir = image.select('B8')
  red = image.select('B4')
  aerosols = image.select('B1')  
  mndvi = (nir.subtract(red)
                      .divide(nir.add(red)
                      .subtract(aerosols.add(aerosols)))
                      .rename('mndvi'))
  return mndvi


# MNDWI - Modified Normalized Water Index = (Green-SWIR)/(Green +SWIR)
def make_mndwi(image, green='B3', swir='B11'):
  green = image.select('B3')
  swir = image.select('B11')
  mndwi = (green.subtract(swir)
                .divide(green.add(swir))
                .rename('mndwi'))
  return mndwi

# Functions to Export to GCS

## Define GCS Locations for Exports

In [ ]:
# DEFINE BUCKET & FOLDER
BUCKET = 'pollutemenot-ai'
FOLDER = "GEE_dataset_final"
#FOLDER = "GEE_dataset_test"

## Function exportRGB (1 image)

In [ ]:
def doExport_RGB(lat, lon, index_danum, size=1000):
  image = s2_sr_median_func(lat, lon, size)
  imageRGB = image.select('B4', 'B3', 'B2')
 
  if size == 1000:
    size_ = "hires"
  else:
    size_ = 'lores'
    
  folder = FOLDER
  
  task = ee.batch.Export.image.toCloudStorage(
      image = imageRGB, 
      description = index_danum,
      bucket = BUCKET,
      fileNamePrefix = folder + '/' + size_ + '/' + index_danum + size_,
      region = square(lat, lon, size).getInfo().get('coordinates'),
      crs = 'EPSG:4326',
      dimensions = "256x256",
      maxPixels = 1E13,
      fileFormat = "GeoTIFF",
      # formatOptions = {cloudOptimized:True }
      )
  task.start()

## Function exportSpectralIndex (4 images)

In [ ]:
def doExport_index(lat, lon, index_danum, size=1000, func=make_ndvi):
  image = s2_sr_median_func(lat, lon, size)
  
  if size == 1000:
    size_ = "hires"
  elif size == 10000:
    size_ = 'lores'
    
  folder = FOLDER
  
  task = ee.batch.Export.image.toCloudStorage(
      image = func(image), 
      description = index_danum + '_' + size_,
      bucket = BUCKET,
      fileNamePrefix = folder + '/' + size_ + '/' + index_danum + size_,
      region = square(lat, lon, size).getInfo().get('coordinates'),
      crs = 'EPSG:4326',
      dimensions = "256x256",
      maxPixels = 1E13,
      fileFormat = "GeoTIFF",
      # formatOptions = {cloudOptimized : True }
      )
  task.start()

## Function exportSRTM (2 images)

In [ ]:
def doExport_srtm(lat, lon, index_danum, size=1000, band="elevation"):
  
  srtm = ee.Image('USGS/SRTMGL1_003')

  if band == "elevation":
    image = srtm.select('elevation')
  else:
    image = ee.Terrain.slope(srtm)

  if size == 1000:
    size_ = "hires"
  elif size == 10000:
    size_ = 'lores'

  folder = FOLDER
  
  task = ee.batch.Export.image.toCloudStorage(
      image = image, 
      description = index_danum + '_' + size_,
      bucket = BUCKET,
      fileNamePrefix = folder + '/' + size_ + '/' + index_danum + size_,
      region = square(lat, lon, size).getInfo().get('coordinates'),
      crs = 'EPSG:4326',
      dimensions = "256x256",
      maxPixels = 1E13,
      fileFormat = "GeoTIFF",
      # formatOptions = { cloudOptimized : True  }
      )
  task.start()

## Function exportJRC

In [ ]:
def doExport_jrc(lat, lon, index_danum, size=1000, band="seasonality"):
  jrc = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
  
  if band == "transition":
    jrc = jrc.select("transition")
  elif band == "max_extent":
    jrc = jrc.select("max_extent")
  else:
    jrc = jrc.select("seasonality")

  if size == 1000:
    size_ = "hires"
  elif size == 10000:
    size_ = 'lores'

  folder = FOLDER
  
  task = ee.batch.Export.image.toCloudStorage(
      image = jrc, 
      description = index_danum + '_' + size_,
      bucket = BUCKET,
      fileNamePrefix = folder + '/' + size_ + '/' + index_danum + size_,
      region = square(lat, lon, size).getInfo().get('coordinates'),
      crs = 'EPSG:4326',
      dimensions = "256x256",
      maxPixels = 1E13,
      fileFormat = "GeoTIFF",
      # formatOptions = { cloudOptimized : True }
      )
  task.start()

# Define ROI

In [ ]:
# Define the patch
def square(lat, lon, size):
  crs_proj = "EPSG:4326"  
  return ee.Geometry.Point([lon, lat], proj=crs_proj).buffer(size).bounds()

# Generate Index Numbers in .csv

In [ ]:
# Generate index numbers in .csv. Will not be required if indexes are added to the .csv after cleaning.
import pandas as pd
datapath = "drive/MyDrive/Data/combined_v2.csv"
df = pd.read_csv(datapath, encoding = "ISO-8859-1")
df["index"] = df.index + 1

# Export to GCS

In [ ]:
import numpy as np

def export_data(index_danum, lat, lon):
  for size in [1000, 10000]:
    doExport_RGB(lat, lon, index_danum + '_RGB_', size)
    doExport_index(lat, lon, index_danum + '_ndvi_', size, make_ndvi)
    doExport_index(lat, lon, index_danum + '_ndwi_', size, make_ndwi)
    doExport_index(lat, lon, index_danum + '_mndvi_', size, make_mndvi)
    doExport_index(lat, lon, index_danum + '_mndwi_', size, make_mndwi)
    doExport_srtm(lat, lon, index_danum + '_elevation_', size, "elevation")
    doExport_srtm(lat, lon, index_danum + '_slope_', size, "slope")
    doExport_jrc(lat, lon, index_danum + '_seasonality_', size, "seasonality")
    doExport_jrc(lat, lon, index_danum + '_transition_', size, "transition")
    doExport_jrc(lat, lon, index_danum + '_max_extent_', size, "max_extent")


In [ ]:
# Assigned begin and end of records for each person
# MADHUKAR: records 1 - 4000
# SHOBHA: records 4000 - 8000
# RADHIKA: records 8000 - 12000
# JOE: 12000 - last

names = ["MADHUKAR", 'SHOBHA', 'RADHIKA', 'JOE']
start = [1, 4000, 8000, 12000]
end = [4000, 8000, 12000, df.shape[0]]
MY_NAME = "RADHIKA"

start_dict = dict(zip(names, start))
end_dict = dict(zip(names, end))

In [ ]:
index_begin = 9000
index_end = 9001

if index_begin >= start_dict[MY_NAME] and index_end <= end_dict[MY_NAME]:
  for count in range(index_begin, index_end):
    if count == index_begin: print("download started from index =", count)
    da_number = df.iloc[count-1]["da_number"]
    latitude = df.iloc[count-1]["latitude"]
    longitude = df.iloc[count-1]["longitude"]
    index = count
    export_data(str(index) + '_' + da_number, latitude, longitude)
    print("Done uploading hires and lores of index =", index)
else:
  print("Please ensure the begin and end is within the interval assigned to you")

print("Woohoo all done!")

download started from index = 9000
Done uploading hires and lores of index = 9000
Woohoo all done!


In [ ]:
task.status()

# Appendix

## Export toDrive

In [ ]:
# THIS WORKS
srtm = ee.Image('USGS/SRTMGL1_003')
# jrc = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
# jrc_bands = jrc.select("seasonality")\
#                 .bandNames().getInfo()
# jrc = jrc.select(jrc_bands)
# srtm = jrc

# lat lon of the site
center = [40.89038, -81.47982]
# reverse coordinates for GEE functions
center_reverse = [center[1], center[0]]


# define roi as a square box having 2 x parameter side
roi = ee.Geometry.Point(center_reverse).bounds().buffer(2000).bounds()

# input colors in the exported tiff for visualization
srtm_tiff = srtm.visualize(min=500 , max=1000, palette=['blue', 'yellow', 'red'])

# retrive scale from the image
scale = srtm.projection().nominalScale()
scale.getInfo()


description = "srtm_2" #srtm.get("system:index").getInfo()
description


task_config = {
    'folder' : 'gee_data',
    'region' : roi.getInfo().get('coordinates'),
    'scale' : scale,
    'maxPixels' : 1E13,
    'fileFormat' : 'GeoTIFF',
    'crs': 'EPSG:4326'

}

task = ee.batch.Export.image.toDrive(srtm_tiff, description, **task_config, fileNamePrefix = "test4")
task.start()
print("Exporting {}".format(description))

Exporting srtm_2


In [ ]:
# THIS WORKS
srtm = ee.Image('USGS/SRTMGL1_003')
center = [47.71011590794007, -123.08330078125]
# reverse coordinates for GEE functions
center_reverse = [center[1], center[0]]

# define roi as a square box having 2 x parameter side
roi = ee.Geometry.Point(center_reverse).bounds().buffer(1000).bounds()

# input colors in the exported tiff for visualization
srtm_tiff = srtm.visualize(min=00 , max=2000, palette=['blue', 'yellow', 'red'])

# retrive scale from the image
scale = srtm.projection().nominalScale()
# scale = scale * ee.ee_number.Number(100)

task = ee.batch.Export.image.toDrive(
    image = srtm_tiff, 
    description = "srtm_0to2000",
    folder = 'gee_data',
    region = roi.getInfo().get('coordinates'),
    scale = 30, 
    crs = 'EPSG:4326',
    maxPixels = 1E13,
    fileFormat = "GeoTIFF"
    )
task.start()
print("Exporting {}".format(description))

Exporting srtm_2


In [ ]:
# image = s2_sr_median.select('B2')
image = ndwi
center = [47.71011590794007, -123.08330078125]
# reverse coordinates for GEE functions
center_reverse = [center[1], center[0]]

# define roi as a square box having 2 x parameter side
roi = ee.Geometry.Point(center_reverse).bounds().buffer(1000).bounds()

# input colors in the exported tiff for visualization
image_tiff = image.visualize(min=0.1, max=0.3, palette=['blue', 'yellow', 'red'])

# retrive scale from the image
scale = image.projection().nominalScale()
# scale = scale * ee.ee_number.Number(100)

task = ee.batch.Export.image.toDrive(
    image = image_tiff, 
    description = "ndwi_-1_to_1",
    folder = 'gee_data',
    region = roi.getInfo().get('coordinates'),
    scale = 20, 
    crs = 'EPSG:4326',
    maxPixels = 1E13,
    fileFormat = "GeoTIFF"
    )
task.start()
print("Exporting {}".format(description))

In [ ]:
# THIS WORKS
srtm = ee.Image('USGS/SRTMGL1_003')
center = [39.18721, -84.78812]
# reverse coordinates for GEE functions
center_reverse = [center[1], center[0]]

# define roi as a square box having 2 x parameter side
roi = ee.Geometry.Point(center_reverse).bounds().buffer(1000).bounds()

# input colors in the exported tiff for visualization
srtm_tiff = srtm.visualize(min=00 , max=2000, palette=['blue', 'yellow', 'red'])

# retrive scale from the image
scale = srtm.projection().nominalScale()
# scale = scale * ee.ee_number.Number(100)

task = ee.batch.Export.image.toDrive(
    image = srtm_tiff, 
    description = "srtm_0to2000",
    folder = 'gee_data',
    region = roi.getInfo().get('coordinates'),
    scale = 30, 
    crs = 'EPSG:4326',
    maxPixels = 1E13,
    fileFormat = "GeoTIFF"
    )
task.start()
print("Exporting {}".format(description))

In [ ]:
image = s2_sr_median_func(lat_id269, lon_id269, 1000)
image.getInfo()

In [ ]:
# jrc.projection().getInfo()

In [ ]:
image = s2_sr_median_func(lat_id269, lon_id269, 1000)
image.bandNames().getInfo()
ndwi = image.normalizedDifference(['B3', 'B8'])
ndwi.getInfo()

In [ ]:
# THIS WORKS!
# image = ee.ImageCollection("COPERNICUS/S2_SR")\
# .filterDate('2018-08-03', '2019-04-03')\
# .filterBounds(ee.Geometry.Point([lon_id269, lat_id269]))\
# .first()

image = s2_sr_median_func(lat_id269, lon_id269, 1000)

# image = (ee.ImageCollection('COPERNICUS/S2_SR')
#         .filterBounds(square_id269)
#         .filterDate(START_DATE, END_DATE)
#         .median())


image = (ee.ImageCollection('COPERNICUS/S2_SR')
            .filterDate('2018-01-01', '2018-12-31')
            .filterBounds(square_id269)
            .median())
# visParams = {bands: ['B4', 'B3', 'B2'], max: 9000};
# Map.addLayer(sen2_filtered, visParams, 'sen2 collection');


image = image.select('B4', 'B3', 'B2', 'B8')


# Create an NDWI image, define visualization parameters and display.
# ndwi = image.normalizedDifference(['B3', 'B8'])
ndwiViz = {'min': -1, 'max': 1, 'palette': ['00FFFF', '0000FF']}

# Mask the non-watery parts of the image, where NDWI < 0.4.
# ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))

# Create visualization layers.
imageRGB = image.visualize(**{'bands': ['B4', 'B3', 'B2'], 'max': 9000, 'min': 0.5})
# ndwiRGB = ndwiMasked.visualize(**{
#   'min': 0,
#   'max': 1,
#   'palette': ['00FFFF', '0000FF']
# })

# task = ee.batch.Export.image.toDrive(
#     image = imageRGB, 
#     description = "imageRGB7",
#     folder = 'gee_data',
#     region = square_id269.getInfo().get('coordinates'),
#     crs = 'EPSG:4326',
#     crs_transform = crs_transform,
#     dimensions = "256x256",
#     maxPixels = 1E13,
#     fileFormat = "GeoTIFF"
#     )


task = ee.batch.Export.image.toDrive(
    image = imageRGB, 
    description = "imageRGB8",
    folder = 'gee_data',
    region = square_id269.getInfo().get('coordinates'),
    scale = 20,
    maxPixels = 1E13,
    fileFormat = "GeoTIFF"
    )

task.start()
# print("Exporting {}".format("ndwiRGB4"))

In [ ]:
# WORKS WONDERFULLY! (here passed scale and roi)
# image = ee.ImageCollection("COPERNICUS/S2_SR")\
# .filterDate('2018-08-03', '2019-04-03')\
# .filterBounds(ee.Geometry.Point([lon_id269, lat_id269]))\
# .first()

image = s2_sr_median_func(lat_id269, lon_id269, 1000)

# image = (ee.ImageCollection('COPERNICUS/S2_SR')
#         .filterBounds(square_id269)
#         .filterDate(START_DATE, END_DATE)
#         .median())


# image = (ee.ImageCollection('COPERNICUS/S2_SR')
#             .filterDate('2018-01-01', '2018-12-31')
#             .filterBounds(square_id269)
#             .median())
# visParams = {bands: ['B4', 'B3', 'B2'], max: 9000};
# Map.addLayer(sen2_filtered, visParams, 'sen2 collection');


image = image.select('B4', 'B3', 'B2', 'B8')


# Create an NDWI image, define visualization parameters and display.
# ndwi = image.normalizedDifference(['B3', 'B8'])
ndwiViz = {'min': -1, 'max': 1, 'palette': ['00FFFF', '0000FF']}

# Mask the non-watery parts of the image, where NDWI < 0.4.
# ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))

# Create visualization layers.
imageRGB = image.visualize(**{'bands': ['B4', 'B3', 'B2'], 'max': 9000, 'min': 0.5})
# ndwiRGB = ndwiMasked.visualize(**{
#   'min': 0,
#   'max': 1,
#   'palette': ['00FFFF', '0000FF']
# })

# task = ee.batch.Export.image.toDrive(
#     image = imageRGB, 
#     description = "imageRGB7",
#     folder = 'gee_data',
#     region = square_id269.getInfo().get('coordinates'),
#     crs = 'EPSG:4326',
#     crs_transform = crs_transform,
#     dimensions = "256x256",
#     maxPixels = 1E13,
#     fileFormat = "GeoTIFF"
#     )


task = ee.batch.Export.image.toDrive(
    image = imageRGB, 
    description = "imageRGB9",
    folder = 'gee_data',
    region = square_id269.getInfo().get('coordinates'),
    scale = 20,
    maxPixels = 1E13,
    fileFormat = "GeoTIFF"
    )

task.start()
# print("Exporting {}".format("ndwiRGB4"))

In [ ]:
# WORKS WONDERFULLY! (here passed dimensions and crs, using crs transform gives black tiff)
# image = ee.ImageCollection("COPERNICUS/S2_SR")\
# .filterDate('2018-08-03', '2019-04-03')\
# .filterBounds(ee.Geometry.Point([lon_id269, lat_id269]))\
# .first()

image = s2_sr_median_func(lat_id269, lon_id269, 1000)

# image = (ee.ImageCollection('COPERNICUS/S2_SR')
#         .filterBounds(square_id269)
#         .filterDate(START_DATE, END_DATE)
#         .median())


# image = (ee.ImageCollection('COPERNICUS/S2_SR')
#             .filterDate('2018-01-01', '2018-12-31')
#             .filterBounds(square_id269)
#             .median())
# visParams = {bands: ['B4', 'B3', 'B2'], max: 9000};
# Map.addLayer(sen2_filtered, visParams, 'sen2 collection');


image = image.select('B4', 'B3', 'B2', 'B8')


# Create an NDWI image, define visualization parameters and display.
# ndwi = image.normalizedDifference(['B3', 'B8'])
ndwiViz = {'min': -1, 'max': 1, 'palette': ['00FFFF', '0000FF']}

# Mask the non-watery parts of the image, where NDWI < 0.4.
# ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))

# Create visualization layers.
imageRGB = image.visualize(**{'bands': ['B4', 'B3', 'B2'], 'max': 9000, 'min': 0.5})
# ndwiRGB = ndwiMasked.visualize(**{
#   'min': 0,
#   'max': 1,
#   'palette': ['00FFFF', '0000FF']
# })

task = ee.batch.Export.image.toDrive(
    image = imageRGB, 
    description = "imageRGB12",
    folder = 'gee_data',
    region = square_id269.getInfo().get('coordinates'),
    crs = 'EPSG:4326',
    # crs_transform = crs_transform,
    dimensions = "256x256",
    maxPixels = 1E13,
    fileFormat = "GeoTIFF"
    )


# task = ee.batch.Export.image.toDrive(
#     image = imageRGB, 
#     description = "imageRGB9",
#     folder = 'gee_data',
#     region = square_id269.getInfo().get('coordinates'),
#     scale = 20,
#     maxPixels = 1E13,
#     fileFormat = "GeoTIFF"
#     )

task.start()
# print("Exporting {}".format("ndwiRGB4"))

In [ ]:
FOLDER

In [ ]:
def doExport_RBG_toDrive(lat, lon, index_danum, size=1000):
  image = s2_sr_median_func(lat, lon, size)
  image = image.select('B4', 'B3', 'B2', 'B8')
  imageRGB = image.visualize(**{'bands': ['B4', 'B3', 'B2'], 'max': 9000, 'min': 0.5})

  if size == 1000:
    size_ = "hires"
  else:
    size_ = 'lores'
  folder = FOLDER
  
  task = ee.batch.Export.image.toDrive(
    image = image, 
    description = index_danum,
    folder = FOLDER,
    region = square(lat, lon, size).getInfo().get('coordinates'),
    crs = 'EPSG:4326',
    # crs_transform = crs_transform,
    dimensions = "256x256",
    maxPixels = 1E13,
    fileFormat = "GeoTIFF"
    
    # we want to preserve this
    # image = imageRGB, 
    # description = "imageRGB12",
    # folder = 'gee_data',
    # region = square_id269.getInfo().get('coordinates'),
    # crs = 'EPSG:4326',
    # # crs_transform = crs_transform,
    # dimensions = "256x256",
    # maxPixels = 1E13,
    # fileFormat = "GeoTIFF"
    )



  task.start()


In [ ]:
def export_data_toDrive(index_danum, lat, lon):
  for size in [1000, 10000]:
    doExport_RBG_toDrive(lat, lon, index_danum + '_RBG_', size)
    # doExport_index(lat, lon, index_danum + '_ndvi_', size, make_ndvi)
    # doExport_index(lat, lon, index_danum + '_ndwi_', size, make_ndwi)
    # doExport_index(lat, lon, index_danum + '_mndvi_', size, make_mndvi)
    # doExport_index(lat, lon, index_danum + '_mndwi_', size, make_mndwi)
    # doExport_srtm(lat, lon, index_danum + '_srtm_', size, None)
    # doExport_jrc(lat, lon, index_danum + '_seasonality_', size, "seasonality")
    # doExport_jrc(lat, lon, index_danum + '_transition_', size, "transition")
    # doExport_jrc(lat, lon, index_danum + '_max_extent_', size, "max_extent")


In [ ]:
# Assigned begin and end of records for each person
# MADHUKAR: records 1 - 4000
# SHOBHA: records 4000 - 8000
# RADHIKA: records 8000 - 12000
# JOE: 12000 - last

names = ["MADHUKAR", 'SHOBHA', 'RADHIKA', 'JOE']
start = [1, 4000, 8000, 12000]
end = [4000, 8000, 12000, df.shape[0]]
MY_NAME = "MADHUKAR"

start_dict = dict(zip(names, start))
end_dict = dict(zip(names, end))

In [ ]:
index_begin = 1
index_end = 2

if index_begin >= start_dict[MY_NAME] and index_end <= end_dict[MY_NAME]:
  for count in range(index_begin, index_end):
    if count == index_begin: print("download started from index =", count)
    da_number = df.iloc[count-1]["da_number"]
    latitude = df.iloc[count-1]["latitude"]
    longitude = df.iloc[count-1]["longitude"]
    index = count
    export_data_toDrive(str(index) + '_' + da_number, latitude, longitude)
    print("Done uploading hires and lores of index =", index)
else:
  print("Please ensure the begin and end is within the interval assigned to you")

print("Woohoo all done!")

## Export toAsset

In [ ]:
def doExport_RBG_toAsset(lat, lon, index_danum, size=1000):
  image = s2_sr_median_func(lat, lon, size)
  image = image.select('B4', 'B3', 'B2', 'B8')
  imageRGB = image.visualize(**{'bands': ['B4', 'B3', 'B2'], 'max': 9000, 'min': 0.5})

  if size == 1000:
    size_ = "hires"
  else:
    size_ = 'lores'
  folder = FOLDER
  
  
  task = ee.batch.Export.image.toAsset(
    image = image, 
    description = index_danum,
    assetId = "users/madhukarreddy/" + str(index_danum),
    region = square(lat, lon, size).getInfo().get('coordinates'),
    crs = 'EPSG:4326',
    # crs_transform = crs_transform,
    dimensions = "256x256",
    maxPixels = 1E13
  )
    
  task.start()


In [ ]:
def doExport_jrc_toAsset(lat, lon, index_danum, size=1000, band="seasonality"):
  jrc = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
  # jrc_bands = jrc.select("seasonality", "transition", "max_extent")\
                # .bandNames().getInfo()
  if band == "transition":
    jrc = jrc.select("transition")
  elif band == "max_extent":
    jrc = jrc.select("max_extent")
  else:
    jrc = jrc.select("seasonality")

  if size == 1000:
    size_ = "hires"
  else:
    size_ = 'lores'
  folder = FOLDER
  
  
  task = ee.batch.Export.image.toAsset(
    image = jrc, 
    description = index_danum + '_' + size_,
    assetId = "users/madhukarreddy/" + str(index_danum),
    region = square(lat, lon, size).getInfo().get('coordinates'),
    crs = 'EPSG:4326',
    # crs_transform = crs_transform,
    dimensions = "256x256",
    maxPixels = 1E13
  )
    
  task.start()


In [ ]:
def doExport_slope_toAsset(lat, lon, index_danum, size=1000, func=None):
  srtm = ee.Image('USGS/SRTMGL1_003')
  image = ee.Terrain.slope(srtm)
  if size == 1000:
    size_ = "hires"
  elif size == 10000:
    size_ = 'lores'
  folder = FOLDER
  
  task = ee.batch.Export.image.toAsset(
      image = image, 
      description = index_danum + '_' + size_,
      assetId = "users/madhukarreddy/" + str(index_danum),
      region = square(lat, lon, size).getInfo().get('coordinates'),
      crs = 'EPSG:4326',
      # crs_transform = crs_transform,
      dimensions = "256x256",
      maxPixels = 1E13
      
      )
  task.start()

In [ ]:
def export_data_toAsset(index_danum, lat, lon):
  for size in [1000, 10000]:
    doExport_RBG_toAsset(lat, lon, index_danum + '_RBG_', size)
    # doExport_index(lat, lon, index_danum + '_ndvi_', size, make_ndvi)
    # doExport_index(lat, lon, index_danum + '_ndwi_', size, make_ndwi)
    # doExport_index(lat, lon, index_danum + '_mndvi_', size, make_mndvi)
    # doExport_index(lat, lon, index_danum + '_mndwi_', size, make_mndwi)
    doExport_slope_toAsset(lat, lon, index_danum + '_slope_', size, None)
    doExport_jrc_toAsset(lat, lon, index_danum + '_seasonality_', size, "seasonality")
    doExport_jrc_toAsset(lat, lon, index_danum + '_transition_', size, "transition")
    doExport_jrc_toAsset(lat, lon, index_danum + '_max_extent_', size, "max_extent")


In [ ]:
# Assigned begin and end of records for each person
# MADHUKAR: records 1 - 4000
# SHOBHA: records 4000 - 8000
# RADHIKA: records 8000 - 12000
# JOE: 12000 - last

names = ["MADHUKAR", 'SHOBHA', 'RADHIKA', 'JOE']
start = [1, 4000, 8000, 12000]
end = [4000, 8000, 12000, df.shape[0]]
MY_NAME = "MADHUKAR"

start_dict = dict(zip(names, start))
end_dict = dict(zip(names, end))

In [ ]:
index_begin = 1
index_end = 2

if index_begin >= start_dict[MY_NAME] and index_end <= end_dict[MY_NAME]:
  for count in range(index_begin, index_end):
    if count == index_begin: print("download started from index =", count)
    da_number = df.iloc[count-1]["da_number"]
    latitude = df.iloc[count-1]["latitude"]
    longitude = df.iloc[count-1]["longitude"]
    index = count
    export_data_toAsset(str(index) + '_' + da_number, latitude, longitude)
    print("Done uploading hires and lores of index =", index)
else:
  print("Please ensure the begin and end is within the interval assigned to you")

print("Woohoo all done!")

download started from index = 1
Done uploading hires and lores of index = 1
Woohoo all done!


In [ ]:
# Assigned begin and end of records for each person
# MADHUKAR: records 1 - 4000
# SHOBHA: records 4000 - 8000
# RADHIKA: records 8000 - 12000
# JOE: 12000 - last

names = ["MADHUKAR", 'SHOBHA', 'RADHIKA', 'JOE']
start = [1, 4000, 8000, 12000]
end = [4000, 8000, 12000, df.shape[0]]
MY_NAME = "MADHUKAR"

start_dict = dict(zip(names, start))
end_dict = dict(zip(names, end))

In [ ]:
index_begin = 1
index_end = 2

if index_begin >= start_dict[MY_NAME] and index_end <= end_dict[MY_NAME]:
  for count in range(index_begin, index_end):
    if count == index_begin: print("download started from index =", count)
    da_number = df.iloc[count-1]["da_number"]
    latitude = df.iloc[count-1]["latitude"]
    longitude = df.iloc[count-1]["longitude"]
    index = count
    export_data(str(index) + '_' + da_number, latitude, longitude)
    print("Done uploading hires and lores of index =", index)
else:
  print("Please ensure the begin and end is within the interval assigned to you")

print("Woohoo all done!")

download started from index = 1
Done uploading hires and lores of index = 1
Woohoo all done!


In [ ]:
task = ee.batch.Export.image.toAsset(
    image = ndwi,
    description = "ndwi_1",
    region = roi.getInfo().get('coordinates'),
    scale = 30,
    assetId = "users/madhukarreddy/example3",
    # dimensions = 256,
    maxPixels = 1e13
  )
task.start()

In [ ]:
task = ee.batch.Export.image.toAsset(
    image=ndwi, 
    assetId='users/madhukarreddy/example7',
    region = roi.getInfo().get('coordinates'),
    scale = 30,
    maxPixels = 1e13
    )
task.start()

In [ ]:
ee.Geometry.Point(center_reverse).bounds().getInfo()

{'coordinates': [[[-81.47982, 40.89038],
   [-81.47982, 40.89038],
   [-81.47982, 40.89038],
   [-81.47982, 40.89038],
   [-81.47982, 40.89038]]],
 'geodesic': False,
 'type': 'Polygon'}

In [ ]:
ee.Geometry.Point(center_reverse).bounds().buffer(1000).getInfo()

{'coordinates': [[[-81.47982, 40.899378865951476],
   [-81.48314533549949, 40.89902066333667],
   [-81.48620587331331, 40.89797457952456],
   [-81.48875792517993, 40.89632391263158],
   [-81.49059833152606, 40.89420009500956],
   [-81.49158063802872, 40.89177221897192],
   [-81.49162674024836, 40.88923356567601],
   [-81.49073307352951, 40.88678621301077],
   [-81.48897086395459, 40.884624949863195],
   [-81.48648043060885, 40.882921777288125],
   [-81.48346000146839, 40.881812228299516],
   [-81.48014993774615, 40.88138459177182],
   [-81.47681362140311, 40.88167289438449],
   [-81.47371652085779, 40.88265419621119],
   [-81.47110509162697, 40.884250413943164],
   [-81.46918718145498, 40.886334527489026],
   [-81.46811549251365, 40.88874067865835],
   [-81.46797541492928, 40.89127736185126],
   [-81.46877820362785, 40.89374266040125],
   [-81.47046004975617, 40.89594031810173],
   [-81.47288713079337, 40.89769536695106],
   [-81.47586624644394, 40.89886806535572],
   [-81.479160198819

In [ ]:
ee.Geometry.Point(center_reverse).bounds().buffer(10000).bounds().getInfo() # this gives a square somehow

{'coordinates': [[[-81.59786893478763, 40.80042587548818],
   [-81.36135975077538, 40.80042587548818],
   [-81.36135975077538, 40.980368659514845],
   [-81.59786893478763, 40.980368659514845],
   [-81.59786893478763, 40.80042587548818]]],
 'geodesic': False,
 'type': 'Polygon'}

In [ ]:
ee.Geometry.Point(center_reverse).bounds().getInfo() # this is just a point

{'coordinates': [[[-81.47982, 40.89038],
   [-81.47982, 40.89038],
   [-81.47982, 40.89038],
   [-81.47982, 40.89038],
   [-81.47982, 40.89038]]],
 'geodesic': False,
 'type': 'Polygon'}